In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import random
import tables
import openslide
from misc.utils import extract_patch, draw_circle , is_nucleus

In [2]:
# Load the H&E image
HE_open = openslide.OpenSlide('Experiment/HE40X/MELANOMA_AA3_40X/Scan1/MELANOMA_AA3_40X_Scan1.qptiff')

# Read H&E cells locations
ROI_size = 4000
for i in range(2,int(55840/ROI_size) - 2): 
    IF_origin = (270 + (i*ROI_size), 150 )
    HE_origin = (3000 + (i*ROI_size), 1000 )
    for j in range(2,int(93600/ROI_size) - 2):
        IF_origin = (IF_origin[0], 150 + (j*ROI_size))
        HE_origin = (HE_origin[0], 1000 + (j*ROI_size))
        if i == 2 and j == 2:
            Mel3_cells_positions = pd.read_csv(f'./locations/3_MEL3_HE_cells_locations_NoOffset_1_{ROI_size}pix_{HE_origin[0]}_{HE_origin[1]}.txt', header = None, names = ['x','y','type'], sep='\t')
            total = Mel3_cells_positions['x'].count()
            continue
        try : 
            df = pd.read_csv(f'./locations/3_MEL3_HE_cells_locations_NoOffset_1_{ROI_size}pix_{HE_origin[0]}_{HE_origin[1]}.txt', header = None, names = ['x','y','type'], sep='\t')
            print(f"Adding {HE_origin[0]}_{HE_origin[1]}, {df['x'].count()}")
            total += df['x'].count()
            Mel3_cells_positions = Mel3_cells_positions.merge(df, how='outer')
        except OSError as e:
            print(e)
            pass
        


Adding 11000_13000, 18474
Adding 11000_17000, 6738
Adding 11000_21000, 15876
Adding 11000_25000, 10351
Adding 11000_29000, 7882
Adding 11000_33000, 10534
Adding 11000_37000, 11690
Adding 11000_41000, 12766
Adding 11000_45000, 7344
Adding 11000_49000, 8077
Adding 11000_53000, 3238
[Errno 2] No such file or directory: './locations/3_MEL3_HE_cells_locations_NoOffset_1_4000pix_11000_57000.txt'
[Errno 2] No such file or directory: './locations/3_MEL3_HE_cells_locations_NoOffset_1_4000pix_11000_61000.txt'
Adding 11000_65000, 0
Adding 11000_69000, 771
Adding 11000_73000, 1693
Adding 11000_77000, 2662
Adding 11000_81000, 312
Adding 15000_9000, 9890
Adding 15000_13000, 6210
Adding 15000_17000, 7795
Adding 15000_21000, 6481
Adding 15000_25000, 4031
Adding 15000_29000, 3805
Adding 15000_33000, 9224
Adding 15000_37000, 14285
Adding 15000_41000, 36
Adding 15000_45000, 0
Adding 15000_49000, 1891
Adding 15000_53000, 4337
Adding 15000_57000, 845
Adding 15000_61000, 571
Adding 15000_65000, 614
Adding 1

In [3]:
Mel3_cells_positions.count()

x       797876
y       797876
type    797876
dtype: int64

In [4]:
#print(f'Total cells count is {total}')
# Count the number of cells per type 
Mel3_cells_positions.groupby(by='type').count()
#Mel3_cells_positions.to_csv('MEL3_HE_cells_locations.txt', header=None, index = None, sep='\t', mode='a')

,x,y
type,,
CD11Cp,18141,18141
CD20p,196629,196629
CD3p,209678,209678
CD56p,14348,14348
CD68p,53896,53896
SOX10p,305184,305184


In [5]:
#phases = ['train','val']
possible =['SOX10p','CD3p','CD20p','CD11Cp','CD68p','CD56p']
# phenotypes that will be present in the database
phenotypes = ['SOX10p','CD3p','CD20p']

# defines the label associated to each class
phenotype_mask = {'SOX10p':0, 'CD3p': 1,'CD20p' : 1}

# number of classes
nclasses = len(set(phenotype_mask.values()))

In [6]:
APC = ['CD11Cp' , 'CD68p']
Immune = ['CD3p','CD20p']

In [7]:
df_filtered = Mel3_cells_positions 

In [8]:
#Splitting the data into training validation and test sets.

size_APC_train = {'CD11Cp':0, 'CD68p': 0}
size_APC_val = {'CD11Cp':0, 'CD68p': 0}
size_APC_test = {'CD11Cp':0, 'CD68p': 0}

size_immune_train = {'CD3p': 0,'CD20p' : 0}
size_immune_val = {'CD3p': 0,'CD20p' : 0}
size_immune_test = {'CD3p': 0,'CD20p' : 0}

size_tumour_train =  {'SOX10p':0}
size_tumour_val = {'SOX10p':0}
size_tumour_test = {'SOX10p':0}


for phenotype in APC:
    size_APC_train [phenotype]=  0#int(Mel3_cells_positions[Mel3_cells_positions['type']==phenotype]["x"].count() * 0.8)
    size_APC_val [phenotype]= 0#int(Mel3_cells_positions[Mel3_cells_positions['type']==phenotype]["x"].count() * 0.1)
    size_APC_test [phenotype]= 0#int(Mel3_cells_positions[Mel3_cells_positions['type']==phenotype]["x"].count() * 0.1)
    
print("APC")
print(f"Train {size_APC_train}")
print(f"Validation {size_APC_val}")
print(f"Test {size_APC_test}")
print("---------------------")

for phenotype in Immune:
    size_immune_train[phenotype]=  125000#int(Mel3_cells_positions[Mel3_cells_positions['type']==phenotype]["x"].count() * 0.8)
    size_immune_val [phenotype]= 10000#int(Mel3_cells_positions[Mel3_cells_positions['type']==phenotype]["x"].count() * 0.1)
    size_immune_test [phenotype]= 10000#int(Mel3_cells_positions[Mel3_cells_positions['type']==phenotype]["x"].count() * 0.1)
    
print("Immune cells")
print(f"Train {size_immune_train}")
print(f"Validation {size_immune_val}")
print(f"Test {size_immune_test}")
print("---------------------")

size_tumour_train ["SOX10p"]=  250000#int(Mel3_cells_positions[Mel3_cells_positions['type']=="SOX10p"]["x"].count() * 0.8)
size_tumour_val ["SOX10p"]= 20000#int(Mel3_cells_positions[Mel3_cells_positions['type']=="SOX10p"]["x"].count() * 0.1)
size_tumour_test ["SOX10p"]= 20000#int(Mel3_cells_positions[Mel3_cells_positions['type']=="SOX10p"]["x"].count() * 0.1)

print("Tumour cells")
print(f"Train {size_tumour_train}")
print(f"Validation {size_tumour_val}")
print(f"Test {size_tumour_test}")

APC
Train {'CD11Cp': 0, 'CD68p': 0}
Validation {'CD11Cp': 0, 'CD68p': 0}
Test {'CD11Cp': 0, 'CD68p': 0}
---------------------
Immune cells
Train {'CD3p': 125000, 'CD20p': 125000}
Validation {'CD3p': 10000, 'CD20p': 10000}
Test {'CD3p': 10000, 'CD20p': 10000}
---------------------
Tumour cells
Train {'SOX10p': 250000}
Validation {'SOX10p': 20000}
Test {'SOX10p': 20000}


In [9]:
# size of the patch
patchsize = 32
dataname = "2_SOX10p_vs_CD3p_CD20p"

In [10]:
print("Extracting train dataset... \n")

# size of the patch
patchsize = 32



# total size 
size = 0
for phenotype in APC : 
    size += size_APC_train [phenotype]
for phenotype in Immune : 
    size += size_immune_train [phenotype]
size += size_tumour_train["SOX10p"]

# open a hdf5_file  in write mode
hdf5_file = tables.open_file(f"dataset/train_{size}_{patchsize}_{nclasses}classes_{dataname}.h5", mode="w", title=f"Database test")

# define the shape of a patch, which is its size and the number of channels
patch_shape = np.array((patchsize, patchsize, 3))

filters = tables.Filters(complevel=6, complib='zlib')

# earray for the patch
hdf5_file.create_earray(hdf5_file.root, "patch", tables.UInt8Atom(), shape=np.append([0], patch_shape),
                            chunkshape=np.append([1], patch_shape), filters=filters)

# earray for the label
hdf5_file.create_earray(hdf5_file.root, "label", tables.UInt16Atom(), shape=[0], chunkshape=[1],
                            filters=filters)

# earray for the classsizes (important for neural network)
hdf5_file.create_earray(hdf5_file.root, "classsizes", tables.UInt16Atom(), shape=[0], chunkshape=[1],
                            filters=filters)

# earray for the image ID
hdf5_file.create_earray(hdf5_file.root, "imgID", tables.UInt32Atom(), shape=[0], chunkshape=[1],
                            filters=filters)


    
for phenotype in phenotypes:
        
    print(f"phenotype: {phenotype}")
    
    if phenotype in Immune:
        
        print("--> Immune \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_immune_train[phenotype]).iterrows():
            
            # extract the patch
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            # append all the information (patch, associated label, ID)
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])

            # remove this sample from the dataframe not to take it back after
            df_filtered.drop(index = index, inplace = True)
        
    elif phenotype in APC: 
        
        print("--> APC \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_APC_train[phenotype]).iterrows():
            
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])
            
            df_filtered.drop(index = index, inplace = True)
    else: 
        
        print("--> Tumour \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_tumour_train[phenotype]).iterrows():
            
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])
            
            df_filtered.drop(index = index, inplace = True)

# add the classsizes for balancing the classes for training
total_APC = 0
for phenotype in APC : 
    total_APC += size_APC_train[phenotype]
#To be improved
total_other = 0
total_immune = 0
for phenotype in Immune : 
    total_immune += size_immune_train[phenotype]
total_other += size_tumour_train["SOX10p"]
hdf5_file.root.classsizes.append([total_immune])
hdf5_file.root.classsizes.append([total_other])
# close the file 
hdf5_file.close()
print("Completed")

Extracting train dataset... 

phenotype: SOX10p
--> Tumour 

phenotype: CD3p
--> Immune 

phenotype: CD20p
--> Immune 



NameError: name 'f' is not defined

In [12]:
print("Extracting val dataset... \n")

# total size 
size = 0
for phenotype in APC : 
    size += size_APC_val [phenotype]
for phenotype in Immune : 
    size += size_immune_val [phenotype]
size += size_tumour_val["SOX10p"]



    
hdf5_file = tables.open_file(f"dataset/val_{size}_{patchsize}_{nclasses}classes_{dataname}.h5", mode="w", title=f"Database test")
    
# define the shape of a patch, which is its size and the number of channels
patch_shape = np.array((patchsize, patchsize, 3))

filters = tables.Filters(complevel=6, complib='zlib')

# earray for the patch
hdf5_file.create_earray(hdf5_file.root, "patch", tables.UInt8Atom(), shape=np.append([0], patch_shape),
                            chunkshape=np.append([1], patch_shape), filters=filters)

# earray for the label
hdf5_file.create_earray(hdf5_file.root, "label", tables.UInt16Atom(), shape=[0], chunkshape=[1],
                            filters=filters)

# earray for the classsizes (important for neural network)
hdf5_file.create_earray(hdf5_file.root, "classsizes", tables.UInt16Atom(), shape=[0], chunkshape=[1],
                            filters=filters)

# earray for the image ID
hdf5_file.create_earray(hdf5_file.root, "imgID", tables.UInt32Atom(), shape=[0], chunkshape=[1],
                            filters=filters)


    
for phenotype in phenotypes:
        
    print(f"phenotype: {phenotype}")
    
    if phenotype in Immune:
        
        print("--> Immune \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_immune_val[phenotype]).iterrows():
            
            # extract the patch
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            # append all the information (patch, associated label, ID)
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])

            # remove this sample from the dataframe not to take it back after
            df_filtered.drop(index = index, inplace = True)
        
    elif phenotype in APC: 
        
        print("--> APC \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_APC_val[phenotype]).iterrows():
            
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])
            
            df_filtered.drop(index = index, inplace = True)
    else: 
        
        print("--> Tumour \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_tumour_val[phenotype]).iterrows():
            
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])
            
            df_filtered.drop(index = index, inplace = True)

# add the classsizes for balancing the classes for training
total_APC = 0
for phenotype in APC : 
    total_APC += size_APC_val[phenotype]
#To be improved
total_other = 0
total_immune = 0
for phenotype in Immune : 
    total_immune += size_immune_val[phenotype]
total_other += size_tumour_val["SOX10p"]

hdf5_file.root.classsizes.append([total_immune])
hdf5_file.root.classsizes.append([total_other])
# close the file 
hdf5_file.close()
print("Completed")

Extracting val dataset... 

phenotype: SOX10p
--> Tumour 

phenotype: CD3p
--> Immune 

phenotype: CD20p
--> Immune 

Completed


In [13]:
print("Extracting test dataset... \n")

# total size 
size = 0
for phenotype in APC : 
    size += size_APC_test [phenotype]
for phenotype in Immune : 
    size += size_immune_test [phenotype]
size += size_tumour_test["SOX10p"]



    
hdf5_file = tables.open_file(f"dataset/test_{size}_{patchsize}_{nclasses}classes_{dataname}.h5", mode="w", title=f"Database test")
    
# define the shape of a patch, which is its size and the number of channels
patch_shape = np.array((patchsize, patchsize, 3))

filters = tables.Filters(complevel=6, complib='zlib')

# earray for the patch
hdf5_file.create_earray(hdf5_file.root, "patch", tables.UInt8Atom(), shape=np.append([0], patch_shape),
                            chunkshape=np.append([1], patch_shape), filters=filters)

# earray for the label
hdf5_file.create_earray(hdf5_file.root, "label", tables.UInt16Atom(), shape=[0], chunkshape=[1],
                            filters=filters)

# earray for the classsizes (important for neural network)
hdf5_file.create_earray(hdf5_file.root, "classsizes", tables.UInt16Atom(), shape=[0], chunkshape=[1],
                            filters=filters)

# earray for the image ID
hdf5_file.create_earray(hdf5_file.root, "imgID", tables.UInt32Atom(), shape=[0], chunkshape=[1],
                            filters=filters)


    
for phenotype in phenotypes:
        
    print(f"phenotype: {phenotype}")
    
    if phenotype in Immune:
        
        print("--> Immune \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_immune_test[phenotype]).iterrows():
            
            # extract the patch
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            # append all the information (patch, associated label, ID)
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])

            # remove this sample from the dataframe not to take it back after
            df_filtered.drop(index = index, inplace = True)
        
    elif phenotype in APC: 
        
        print("--> APC \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_APC_test[phenotype]).iterrows():
            
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])
            
            df_filtered.drop(index = index, inplace = True)
    else: 
        
        print("--> Tumour \n")
        
        for index, row in df_filtered[df_filtered['type']==phenotype].sample(size_tumour_test[phenotype]).iterrows():
            
            patch = extract_patch(HE_open, int(row['x']), int(row['y']), patchsize)
            
            hdf5_file.root.patch.append(patch[None, ::])
            hdf5_file.root.label.append([phenotype_mask[phenotype]])
            hdf5_file.root.imgID.append([index])
            
            df_filtered.drop(index = index, inplace = True)


# add the classsizes for balancing the classes for training
total_APC = 0
for phenotype in APC : 
    total_APC += size_APC_test[phenotype]
    
#To be improved
total_other = 0
total_immune = 0
for phenotype in Immune : 
    total_immune += size_immune_test[phenotype]
total_other += size_tumour_test["SOX10p"]

hdf5_file.root.classsizes.append([total_immune])
hdf5_file.root.classsizes.append([total_other])
# close the file 
hdf5_file.close()
print("Completed")

Extracting test dataset... 

phenotype: SOX10p
--> Tumour 

phenotype: CD3p
--> Immune 

phenotype: CD20p
--> Immune 

Completed
